In [1]:
!pip install -q "sagemaker>=2.70.0" "transformers==4.11.0" --upgrade

!pip install -q "datasets==1.13" --upgrade


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.25.73 requires botocore==1.27.72, but you have botocore 1.27.73 which is incompatible.
aiobotocore 2.0.1 requires botocore<1.22.9,>=1.22.8, but you have botocore 1.27.73 which is incompatible.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install -q torch

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import random
import pandas as pd

In [4]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

# Data Preprocessing

In [5]:
dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))

In [6]:
documents = dataset.data
labels = dataset.target
df=pd.DataFrame({"documents":documents,"labels":labels})

In [7]:
df

,documents,labels
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,13
18842,\nNot in isolated ground recepticles (usually ...,12
18843,I just installed a DX2-66 CPU in a clone mothe...,3
18844,\nWouldn't this require a hyper-sphere. In 3-...,1


In [8]:
import re,string

def text_cleaner(text):

    '''some text cleaning method'''

    text = text.lower()

    text = re.sub('\[.*?\]', '', text)

    text = re.sub('https?://\S+|www\.\S+', '', text)

    text = re.sub('<.*?>+', '', text)

    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

    text = re.sub('\n', '', text)

    text = re.sub('\w*\d\w*', '', text)

    return text

In [9]:
df["documents"]=df["documents"].apply(text_cleaner)

In [10]:
df=df.head(1100)

In [11]:
df.shape

(1100, 2)

In [12]:
df

,documents,labels
0,i am sure some bashers of pens fans are pretty...,10
1,my brother is in the market for a highperforma...,3
2,\tfinally you said what you dream about medite...,17
3,thinkits the scsi card doing the dma transfers...,3
4,i have an old jasmine drive which i cannot...,4
...,...,...
1095,i am hoping to produce the first update of the...,9
1096,in article manesmagpielinknetcom steve i...,16
1097,sorry malcolm but i rather believe jesus than...,19
1098,i have placed a new release of my axe editor i...,5


In [13]:
df = df.sample(frac = 1)

In [14]:
df

,documents,labels
232,oh and us with the big degrees dont got imagin...,14
201,i have version which i believe was needed for...,4
474,sutcliffe gives up hrs gonzales palmer and ...,9
722,\tok ill admit it i cant find a quote with my...,17
378,recently i have been getting a cmos checksum e...,3
...,...,...
666,not necessarily it could mean that or it coul...,9
54,dont worry about this theyll drop you like a ...,7
714,april eff analysis of clinton privacy and se...,11
501,i have the same problem someone suggested it m...,2


In [15]:
rows = df.shape[0]
train = int(.8* rows)
test = rows-train

In [16]:
df[:train].shape,df[train:].shape

((880, 2), (220, 2))

In [17]:
df[:train].to_csv('train.csv'
                          ,index=False)

In [18]:
# Write Validation Set
df[train:].to_csv('test.csv'
                          ,index=False)

In [19]:
df

,documents,labels
232,oh and us with the big degrees dont got imagin...,14
201,i have version which i believe was needed for...,4
474,sutcliffe gives up hrs gonzales palmer and ...,9
722,\tok ill admit it i cant find a quote with my...,17
378,recently i have been getting a cmos checksum e...,3
...,...,...
666,not necessarily it could mean that or it coul...,9
54,dont worry about this theyll drop you like a ...,7
714,april eff analysis of clinton privacy and se...,11
501,i have the same problem someone suggested it m...,2


In [21]:
df[["labels"]]

,labels
232,14
201,4
474,9
722,17
378,3
...,...
666,9
54,7
714,11
501,2


# Sagemaker

In [5]:
import sagemaker
from sagemaker import get_execution_role
# from sagemaker.sklearn.estimator import SKLearn
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name
import os
import boto3

In [6]:
region

'us-east-2'

In [7]:
bucket_name = 'agtimeseries'

training_folder = r'mme-hf-folder/train'
test_folder = r'mme-hf-folder/test'
model_folder1 = r'mme-hf-folder/model1/'
script_code_folder1=r"mme-hf-folder/scriptcode1"

model_folder2 = r'mme-hf-folder/model2/'
script_code_folder2=r"mme-hf-folder/scriptcode2"


model_folder3 = r'mme-hf-folder/model3/'
script_code_folder3=r"mme-hf-folder/scriptcode3"

training_data_loc = r's3://' + bucket_name + r'/' + training_folder
testing_data_loc = r's3://' + bucket_name + r'/' + test_folder

model_data_loc1 = r's3://' + bucket_name + r'/' + model_folder1
script_code_loc1=r's3://' + bucket_name + r'/' + script_code_folder1


model_data_loc2 = r's3://' + bucket_name + r'/' + model_folder2
script_code_loc2=r's3://' + bucket_name + r'/' + script_code_folder2

model_data_loc3 = r's3://' + bucket_name + r'/' + model_folder3
script_code_loc3=r's3://' + bucket_name + r'/' + script_code_folder3

In [8]:
training_data_loc

's3://agtimeseries/mme-hf-folder/train'

In [9]:
train_file="train.csv"
test_file="test.csv"

In [10]:
sagemaker_session.upload_data(train_file,
                              bucket=bucket_name, 
                              key_prefix=training_folder)

's3://agtimeseries/mme-hf-folder/train/train.csv'

In [11]:
sagemaker_session.upload_data(test_file, 
                              bucket=bucket_name, 
                              key_prefix=test_folder)

's3://agtimeseries/mme-hf-folder/test/test.csv'

In [12]:
model_data_loc1

's3://agtimeseries/mme-hf-folder/model1/'

In [13]:
from sagemaker.huggingface import HuggingFace,TrainingCompilerConfig

In [14]:
compiler_config=TrainingCompilerConfig()

# Estimator: 1 

In [15]:
hyperparameters={'epochs': 1,                                    # number of training epochs
                 'train_batch_size': 8,                         # batch size for training
                 'eval_batch_size': 8,                          #batch size for testing
                 "model_id" :'bert-base-uncased'                #pretrained model name from hf
                }

In [16]:
huggingface_estimator1 = HuggingFace(
    entry_point          = 'script3.py',       # script code to run the model      
    source_dir           = 'code/',
    instance_type        = 'ml.p3.2xlarge',    # instances type used for the training job
    instance_count       = 1,                  # the number of instances used for training
    base_job_name        = "bert-model",       # the name of the training job     
    role                 = role,               # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.11.0',           # the transformers version used in the training job
    pytorch_version      = '1.9.0',            # the pytorch_version version used in the training job
    py_version           = 'py38',             # the python version used in the training job
    compiler_config      = compiler_config,    # the compiler configuration used in the training job
    hyperparameters      = hyperparameters,    # the hyperparameter used for running the training job
    disable_profiler     = True,               # whether to disable the profiler during training used to gain maximum performance
    debugger_hook_config = False,              # whether to enable the debugger hook during training used to gain maximum performance
    output_path          = model_data_loc1,     # s3 location where the final fine-tuned model will be saved 
    code_location        = script_code_loc1     # s3 location where the estimator will push the script code
)

In [17]:
data={'training': training_data_loc, 'testing': testing_data_loc}

In [18]:
huggingface_estimator1.fit(data)

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: bert-model-2023-06-04-07-51-04-231


2023-06-04 07:51:04 Starting - Starting the training job...
2023-06-04 07:51:29 Starting - Preparing the instances for training......
2023-06-04 07:52:30 Downloading - Downloading input data...
2023-06-04 07:52:50 Training - Downloading the training image...........................
2023-06-04 07:57:36 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-06-04 07:58:00,360 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-06-04 07:58:00,388 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-06-04 07:58:00,391 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-06-04 07:58:00,661 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {
        "sagemaker_training_compiler_debug_mode"

Downloading:  25%|██▌       | 105M/420M [00:01<00:04, 74.7MB/s]
Downloading:  27%|██▋       | 113M/420M [00:01<00:04, 74.9MB/s]
Downloading:  29%|██▊       | 120M/420M [00:01<00:04, 74.9MB/s]
Downloading:  30%|███       | 127M/420M [00:01<00:04, 74.8MB/s]
Downloading:  32%|███▏      | 134M/420M [00:01<00:04, 74.9MB/s]
Downloading:  34%|███▎      | 141M/420M [00:02<00:03, 74.4MB/s]
Downloading:  35%|███▌      | 148M/420M [00:02<00:03, 74.3MB/s]
Downloading:  37%|███▋      | 155M/420M [00:02<00:03, 74.2MB/s]
Downloading:  39%|███▊      | 162M/420M [00:02<00:03, 74.2MB/s]
Downloading:  40%|████      | 170M/420M [00:02<00:03, 74.3MB/s]
Downloading:  42%|████▏     | 177M/420M [00:02<00:03, 74.4MB/s]
Downloading:  44%|████▍     | 184M/420M [00:02<00:03, 74.5MB/s]
Downloading:  45%|████▌     | 191M/420M [00:02<00:03, 74.4MB/s]
Downloading:  47%|████▋     | 198M/420M [00:02<00:03, 74.4MB/s]
Downloading:  49%|████▉     | 205M/420M [00:02<00:03, 74.4MB/s]
Downloading:  51%|█████     | 212M/420M 

100%|██████████| 107/107 [02:26<00:00,  3.18s/it]
***** Running Evaluation *****
***** Running Evaluation *****
  Num examples = 216
Num examples = 216
  Batch size = 8
Batch size = 8
100%|██████████| 27/27 [00:02<00:00,  9.50it/s]#033[A
#033[A
{'eval_loss': 2.5112102031707764, 'eval_accuracy': 0.3194444444444444, 'eval_runtime': 46.5581, 'eval_samples_per_second': 4.639, 'eval_steps_per_second': 0.58, 'epoch': 1.0}
#015100%|██████████| 27/27 [00:02<00:00,  9.50it/s]#033[A
#033[A
Saving model checkpoint to ./results/checkpoint-107
Saving model checkpoint to ./results/checkpoint-107
Configuration saved in ./results/checkpoint-107/config.json
Configuration saved in ./results/checkpoint-107/config.json
Model weights saved in ./results/checkpoint-107/pytorch_model.bin
Model weights saved in ./results/checkpoint-107/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-107/tokenizer_config.json
tokenizer config file saved in ./results/checkpoint-107/tokenizer_config.json
Spe


2023-06-04 08:01:52 Uploading - Uploading generated training model
2023-06-04 08:02:53 Completed - Training job completed
Training seconds: 624
Billable seconds: 624


In [92]:
huggingface_estimator1.model_data

's3://agtimeseries/mme-hf-folder/model1/bert-model-2023-06-04-07-51-04-231/output/model.tar.gz'

In [93]:
huggingface_estimator1

# Estimator: 2

In [25]:
hyperparameters={'epochs': 1,                                    # number of training epochs
                 'train_batch_size': 8,                         # batch size for training
                 'eval_batch_size': 8,                          #batch size for testing
                 "model_id" :'bert-base-uncased'                #pretrained model name from hf
                }

In [26]:
huggingface_estimator2 = HuggingFace(
    entry_point          = 'script3.py',       # script code to run the model      
    source_dir           = 'code/',
    instance_type        = 'ml.p3.2xlarge',    # instances type used for the training job
    instance_count       = 1,                  # the number of instances used for training
    base_job_name        = "bert-model",       # the name of the training job     
    role                 = role,               # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.11.0',           # the transformers version used in the training job
    pytorch_version      = '1.9.0',            # the pytorch_version version used in the training job
    py_version           = 'py38',             # the python version used in the training job
    compiler_config      = compiler_config,    # the compiler configuration used in the training job
    hyperparameters      = hyperparameters,    # the hyperparameter used for running the training job
    disable_profiler     = True,               # whether to disable the profiler during training used to gain maximum performance
    debugger_hook_config = False,              # whether to enable the debugger hook during training used to gain maximum performance
    output_path          = model_data_loc2,     # s3 location where the final fine-tuned model will be saved 
    code_location        = script_code_loc2     # s3 location where the estimator will push the script code
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [27]:
data={'training': training_data_loc, 'testing': testing_data_loc}

In [28]:
huggingface_estimator2.fit(data)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: bert-model-2023-06-04-08-12-01-845


Using provided s3_resource
2023-06-04 08:12:02 Starting - Starting the training job...
2023-06-04 08:12:28 Starting - Preparing the instances for training......
2023-06-04 08:13:30 Downloading - Downloading input data...
2023-06-04 08:13:50 Training - Downloading the training image..............................
2023-06-04 08:18:51 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-06-04 08:19:13,709 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-06-04 08:19:13,736 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-06-04 08:19:13,739 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-06-04 08:19:13,996 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {
        "sagemaker_

Using XLA device
Using XLA device
***** Running training *****
***** Running training *****
  Num examples = 853
Num examples = 853
  Num Epochs = 1
  Instantaneous batch size per device = 8
Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 107
Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 107
0%|          | 0/107 [00:00<?, ?it/s]
[2023-06-04 08:19:28.524 algo-1:27 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-06-04 08:19:28.633 algo-1:27 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
100%|██████████| 107/107 [02:25<00:00,  3.12s/it]
***** Running Evaluation *****
***** Running Evaluation *****
Num examples = 216
  Batch size = 8
Num examples = 216
  Batch size = 8
100%|██

100%|██████████| 27/27 [00:02<00:00,  9.95it/s]
Saving model checkpoint to /opt/ml/model
Saving model checkpoint to /opt/ml/model
Configuration saved in /opt/ml/model/config.json
Configuration saved in /opt/ml/model/config.json
Model weights saved in /opt/ml/model/pytorch_model.bin
Model weights saved in /opt/ml/model/pytorch_model.bin
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
Special tokens file saved in /opt/ml/model/special_tokens_map.json
Special tokens file saved in /opt/ml/model/special_tokens_map.json
2023-06-04 08:22:51,267 sagemaker-training-toolkit INFO     Waiting for the process to finish and give a return code.
2023-06-04 08:22:51,268 sagemaker-training-toolkit INFO     Done waiting for a return code. Received 0 from exiting process.
2023-06-04 08:22:51,268 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2023-06-04 08:23:02 Uploading - Uploading generated training mo

# Estimator: 3

In [29]:
hyperparameters={'epochs': 1,                                    # number of training epochs
                 'train_batch_size': 8,                         # batch size for training
                 'eval_batch_size': 8,                          #batch size for testing
                 "model_id" :'bert-base-uncased'                #pretrained model name from hf
                }

In [30]:
huggingface_estimator3 = HuggingFace(
    entry_point          = 'script3.py',       # script code to run the model      
    source_dir           = 'code/',
    instance_type        = 'ml.p3.2xlarge',    # instances type used for the training job
    instance_count       = 1,                  # the number of instances used for training
    base_job_name        = "bert-model",       # the name of the training job     
    role                 = role,               # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.11.0',           # the transformers version used in the training job
    pytorch_version      = '1.9.0',            # the pytorch_version version used in the training job
    py_version           = 'py38',             # the python version used in the training job
    compiler_config      = compiler_config,    # the compiler configuration used in the training job
    hyperparameters      = hyperparameters,    # the hyperparameter used for running the training job
    disable_profiler     = True,               # whether to disable the profiler during training used to gain maximum performance
    debugger_hook_config = False,              # whether to enable the debugger hook during training used to gain maximum performance
    output_path          = model_data_loc3,     # s3 location where the final fine-tuned model will be saved 
    code_location        = script_code_loc3     # s3 location where the estimator will push the script code
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [31]:
data={'training': training_data_loc, 'testing': testing_data_loc}

In [32]:
huggingface_estimator3.fit(data)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: bert-model-2023-06-04-08-24-23-178


2023-06-04 08:24:23 Starting - Starting the training job...
2023-06-04 08:24:49 Starting - Preparing the instances for training......
2023-06-04 08:25:52 Downloading - Downloading input data...
2023-06-04 08:26:12 Training - Downloading the training image..............................
2023-06-04 08:31:24 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-06-04 08:31:51,286 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-06-04 08:31:51,313 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-06-04 08:31:51,315 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-06-04 08:31:51,562 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {
        "sagemaker_training_compiler_debug_m

Using XLA device
Using XLA device
***** Running training *****
Num examples = 853
***** Running training *****
  Num examples = 853
Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
Total optimization steps = 107
Total optimization steps = 107
0%|          | 0/107 [00:00<?, ?it/s]
[2023-06-04 08:32:06.287 algo-1:27 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-06-04 08:32:06.401 algo-1:27 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
100%|██████████| 107/107 [02:25<00:00,  3.08s/it]
***** Running Evaluation *****
***** Running Evaluation *****
  Num examples = 216
Num examples = 216
  Batch size = 8
Batch size = 8
100%|██████

Training completed. Do not forget to share your model on huggingface.co/models =)
Loading best model from ./results/checkpoint-107 (score: 2.345978260040283).
Training completed. Do not forget to share your model on huggingface.co/models =)
Loading best model from ./results/checkpoint-107 (score: 2.345978260040283).
{'train_runtime': 197.1193, 'train_samples_per_second': 4.327, 'train_steps_per_second': 0.543, 'train_loss': 2.7093560049466996, 'epoch': 1.0}
100%|██████████| 107/107 [03:17<00:00,  1.84s/it]
***** Running Evaluation *****
***** Running Evaluation *****
  Num examples = 216
Num examples = 216
  Batch size = 8
Batch size = 8
100%|██████████| 27/27 [00:02<00:00,  9.76it/s]
Saving model checkpoint to /opt/ml/model
Saving model checkpoint to /opt/ml/model
Configuration saved in /opt/ml/model/config.json
Configuration saved in /opt/ml/model/config.json
Model weights saved in /opt/ml/model/pytorch_model.bin
Model weights saved in /opt/ml/model/pytorch_model.bin
tokenizer config

# Endpoint Name and update the location of both models

In [143]:
import boto3
from time import gmtime, strftime
DATA_PREFIX = "DEMO_HF_MME_Ver1"
CUR = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
MULTI_MODEL_ARTIFACTS = "multi_model_artifacts"+ "-" + CUR

CUR = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
ENDPOINT_NAME = "mme-hf-POC-V1" + "-" + CUR

MODEL_NAME = ENDPOINT_NAME

In [144]:
ENDPOINT_NAME

'mme-hf-POC-V1-2023-06-05-13-44-06'

In [116]:
multimodels_path = f"s3://{bucket_name}/{DATA_PREFIX}/{MULTI_MODEL_ARTIFACTS}/"

In [117]:
multimodels_path

's3://agtimeseries/DEMO_HF_MME_Ver1/multi_model_artifacts-2023-06-05-10-14-02/'

In [118]:
estimators=[]
estimators.append(huggingface_estimator1)
estimators.append(huggingface_estimator2)
estimators.append(huggingface_estimator3)

In [119]:
s3 = boto3.resource('s3')

In [120]:
# copy the model from the base location to the new location where both of them will be saved
for est in estimators:
    artifact_path = est.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]
    model_name = artifact_path.split("/")[-4] + ".tar.gz"
    prefix=multimodels_path.split(bucket_name)[1][1:]
    model_path=prefix+model_name
    s3.Object(bucket_name,model_path).copy_from(CopySource=artifact_path.split("//")[1])   

# Create Model, Endpoint Config, Endpoint

In [131]:
multimodels_path

's3://agtimeseries/DEMO_HF_MME_Ver1/multi_model_artifacts-2023-06-05-10-14-02/'

In [145]:
sm_client = boto3.client("sagemaker")
# image_uri will depend on the region
image_uri = "763104351884.dkr.ecr.us-east-2.amazonaws.com/huggingface-pytorch-inference:1.9.0-transformers4.11.0-cpu-py38-ubuntu20.04"
primary_container = {
    'Image': image_uri,
    'Mode': 'MultiModel',
    'ModelDataUrl': multimodels_path,
    'Environment': {
        'SAGEMAKER_PROGRAM': 'inference.py',
        'SAGEMAKER_REGION': region,
        'SAGEMAKER_SUBMIT_DIRECTORY': multimodels_path
    }
}

create_model_response = sm_client.create_model(ModelName = ENDPOINT_NAME,
                                              ExecutionRoleArn = get_execution_role(),
                                              PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

arn:aws:sagemaker:us-east-2:831536787935:model/mme-hf-POC-V1-2023-06-05-13-44-06


In [146]:
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName =ENDPOINT_NAME,
    ProductionVariants=[
        {
        'InstanceType':'ml.m5.2xlarge',
        'InitialInstanceCount':1,
        'ModelName': ENDPOINT_NAME,
        'VariantName':'AllTraffic',
        'InitialVariantWeight':1
        }
    ])

print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

Endpoint configuration arn:  arn:aws:sagemaker:us-east-2:831536787935:endpoint-config/mme-hf-POC-V1-2023-06-05-13-44-06


In [147]:
endpoint_params = {
    'EndpointName': ENDPOINT_NAME,
    'EndpointConfigName':ENDPOINT_NAME,
}
endpoint_response = sm_client.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

EndpointArn = arn:aws:sagemaker:us-east-2:831536787935:endpoint/mme-hf-POC-V1-2023-06-05-13-44-06


In [126]:
waiter=sm_client.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=ENDPOINT_NAME)

In [135]:
ENDPOINT_NAME

'mme-hf-POC-V1-2023-06-05-11-44-37'

In [148]:
import boto3

# Let us define a client to play with autoscaling options
asg_client = boto3.client('application-autoscaling') # Common class representing Application Auto Scaling for SageMaker amongst other services

# the resource type is variant and the unique identifier is the resource ID.
# Example: endpoint/my-bert-fine-tuned/variant/AllTraffic .
resource_id=f"endpoint/{ENDPOINT_NAME}/variant/AllTraffic"

# scaling configuration
response = asg_client.register_scalable_target(
    ServiceNamespace='sagemaker', #
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=1,
    MaxCapacity=3
)


response = asg_client.put_scaling_policy(
    PolicyName=f'CPUUtil-ScalingPolicy-{ENDPOINT_NAME}',
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 50.0, # threshold
        'CustomizedMetricSpecification':
        {
            'MetricName': 'CPUUtilization',
            'Namespace': '/aws/sagemaker/Endpoints',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': ENDPOINT_NAME },
                {'Name': 'VariantName','Value': 'AllTraffic'}
            ],
            'Statistic': 'Average', # Possible - 'Statistic': 'Average'|'Minimum'|'Maximum'|'SampleCount'|'Sum'
            'Unit': 'Percent'
        },
        'ScaleInCooldown': 120, # duration until scale in
        'ScaleOutCooldown': 5 # duration between scale out
    }
)


# Endpoint Test

In [136]:
sen1='the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below'
sen2='the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below'

In [137]:
v=[{"text":sen1,"id":1},{"text":sen2,"id":2}]

In [138]:
import boto3
import json
client = boto3.client('sagemaker-runtime')

In [139]:
import boto3
import json
client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
EndpointName=ENDPOINT_NAME,
ContentType="application/json",
Accept="application/json",
TargetModel="model2.tar.gz",
Body=json.dumps(v)
)
print(response['Body'].read().decode())

[
  {
    "score": 0.09118082374334335,
    "Categ": 17,
    "pred": "l1",
    "probabs": {
      "0": 0.04393492639064789,
      "1": 0.047898419201374054,
      "2": 0.035997066646814346,
      "3": 0.04286786913871765,
      "4": 0.024708978831768036,
      "5": 0.04014800488948822,
      "6": 0.04083900526165962,
      "7": 0.06763872504234314,
      "8": 0.07068825513124466,
      "9": 0.05222117155790329,
      "10": 0.07997382432222366,
      "11": 0.026193739846348763,
      "12": 0.03541635721921921,
      "13": 0.07945353537797928,
      "14": 0.04111085459589958,
      "15": 0.04082481935620308,
      "16": 0.04097103700041771,
      "17": 0.09118082374334335,
      "18": 0.044053815305233,
      "19": 0.053878821432590485
    },
    "text": "the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below",
    "id": 1
  },


In [140]:
import boto3
import json
client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
EndpointName=ENDPOINT_NAME,
ContentType="application/json",
Accept="application/json",
TargetModel="model1.tar.gz",
Body=json.dumps(v)
)
print(response['Body'].read().decode())

[
  {
    "score": 0.0762203261256218,
    "Categ": 12,
    "pred": "m",
    "probabs": {
      "0": 0.053064700216054916,
      "1": 0.03383437171578407,
      "2": 0.023381473496556282,
      "3": 0.06128808110952377,
      "4": 0.04226318374276161,
      "5": 0.0382646769285202,
      "6": 0.04201752692461014,
      "7": 0.06270086020231247,
      "8": 0.07121488451957703,
      "9": 0.05647360533475876,
      "10": 0.057041436433792114,
      "11": 0.02902052365243435,
      "12": 0.0762203261256218,
      "13": 0.061086080968379974,
      "14": 0.0694165751338005,
      "15": 0.048459939658641815,
      "16": 0.0401015505194664,
      "17": 0.04761810600757599,
      "18": 0.03600163385272026,
      "19": 0.05053044855594635
    },
    "text": "the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below",
    "id": 1
  },
  {


In [141]:
import boto3
import json
client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
EndpointName=ENDPOINT_NAME,
ContentType="application/json",
Accept="application/json",
TargetModel="model2.tar.gz",
Body=json.dumps(v)
)
print(response['Body'].read().decode())

[
  {
    "score": 0.09118082374334335,
    "Categ": 17,
    "pred": "l1",
    "probabs": {
      "0": 0.04393492639064789,
      "1": 0.047898419201374054,
      "2": 0.035997066646814346,
      "3": 0.04286786913871765,
      "4": 0.024708978831768036,
      "5": 0.04014800488948822,
      "6": 0.04083900526165962,
      "7": 0.06763872504234314,
      "8": 0.07068825513124466,
      "9": 0.05222117155790329,
      "10": 0.07997382432222366,
      "11": 0.026193739846348763,
      "12": 0.03541635721921921,
      "13": 0.07945353537797928,
      "14": 0.04111085459589958,
      "15": 0.04082481935620308,
      "16": 0.04097103700041771,
      "17": 0.09118082374334335,
      "18": 0.044053815305233,
      "19": 0.053878821432590485
    },
    "text": "the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below",
    "id": 1
  },


In [142]:
%%time
import boto3
import json
client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
EndpointName=ENDPOINT_NAME,
ContentType="application/json",
Accept="application/json",
TargetModel="model3.tar.gz",
Body=json.dumps(v)
)
print(response['Body'].read().decode())

[
  {
    "score": 0.10249396413564682,
    "Categ": 15,
    "pred": "j1",
    "probabs": {
      "0": 0.05833074823021889,
      "1": 0.03819238767027855,
      "2": 0.03585965931415558,
      "3": 0.05405878275632858,
      "4": 0.0336628258228302,
      "5": 0.03182928264141083,
      "6": 0.035747431218624115,
      "7": 0.05438819155097008,
      "8": 0.06668586283922195,
      "9": 0.04516938328742981,
      "10": 0.050032839179039,
      "11": 0.03866000846028328,
      "12": 0.06522325426340103,
      "13": 0.06857302039861679,
      "14": 0.053612589836120605,
      "15": 0.10249396413564682,
      "16": 0.03439391031861305,
      "17": 0.0440681166946888,
      "18": 0.04143171384930611,
      "19": 0.04758596420288086
    },
    "text": "the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below",
    "id": 1
  },
  {
 

In [52]:
def lambda_handler(event, context="None"):
        all=[]
#         event=eval(event["body"])
        print(event)
        event=json.loads(json.dumps(event))
        response1 = client.invoke_endpoint(EndpointName="mme-hf-POC-V1-2023-06-04-09-06-29",ContentType="application/json",Accept="application/json",TargetModel="model1.tar.gz",Body=json.dumps(v))
        response2 = client.invoke_endpoint(EndpointName="mme-hf-POC-V1-2023-06-04-09-06-29",ContentType="application/json",Accept="application/json",TargetModel="model2.tar.gz",Body=json.dumps(v))
        response3 = client.invoke_endpoint(EndpointName="mme-hf-POC-V1-2023-06-04-09-06-29",ContentType="application/json",Accept="application/json",TargetModel="model3.tar.gz",Body=json.dumps(v))
        value1=json.loads(response1['Body'].read().decode())
        value2=json.loads(response2['Body'].read().decode())
        value3=json.loads(response3['Body'].read().decode())
        all.extend(value1)
        all.extend(value2)
        all.extend(value3)
        return all

In [54]:
v=lambda_handler(v)

[{'text': 'the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below', 'id': 1}, {'text': 'the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below', 'id': 2}]


In [56]:
len(v)

6

In [58]:
v=[{"score":0.0786677822470665,"probabs":{"11":0.0734185054898262,"12":0.053138475865125656,"13":0.0645674467086792,"14":0.04427026957273483,"15":0.03951818495988846,"16":0.05541767179965973,"17":0.05795884504914284,"18":0.044276174157857895,"19":0.05284232273697853,"0":0.0347941555082798,"1":0.047643449157476425,"2":0.058559220284223557,"3":0.03778599575161934,"4":0.05978680029511452,"5":0.0786677822470665,"6":0.03158474713563919,"7":0.035702262073755264,"8":0.043942879885435104,"9":0.051764875650405884,"10":0.03435995429754257},"Categ":5,"pred":"f","text":" SEE ACCURACY IN ADULTS ?","id":31},{"score":0.07641265541315079,"probabs":{"11":0.06905900686979294,"12":0.04097370803356171,"13":0.07641265541315079,"14":0.04322277754545212,"15":0.057789236307144165,"16":0.04455627128481865,"17":0.06774531304836273,"18":0.04826144874095917,"19":0.0570475235581398,"0":0.04471467435359955,"1":0.06566658616065979,"2":0.0529228039085865,"3":0.04011021554470062,"4":0.047212518751621246,"5":0.055869825184345245,"6":0.03973971679806709,"7":0.028527729213237762,"8":0.03630654886364937,"9":0.04106844216585159,"10":0.04279303178191185},"Categ":13,"pred":"n","text":"50-56% of patients maintained remission (das28-hscrp 2.6)","id":32},{"score":0.07514258474111557,"probabs":{"11":0.05765023082494736,"12":0.05755647271871567,"13":0.07514258474111557,"14":0.04839545488357544,"15":0.046911340206861496,"16":0.04532558470964432,"17":0.05318426713347435,"18":0.041376255452632904,"19":0.06190388649702072,"0":0.042031336575746536,"1":0.05209396407008171,"2":0.049786075949668884,"3":0.04420416057109833,"4":0.06421200186014175,"5":0.0637398511171341,"6":0.04055355116724968,"7":0.03757360205054283,"8":0.036787837743759155,"9":0.04347537085413933,"10":0.03809618949890137},"Categ":13,"pred":"n","text":"Safety profile of baricitinib: All Bari dataset","id":34},{"score":0.07422996312379837,"probabs":{"11":0.06959327310323715,"12":0.06562454253435135,"13":0.07233278453350067,"14":0.04503995552659035,"15":0.040700286626815796,"16":0.04539260268211365,"17":0.04823519289493561,"18":0.03677678853273392,"19":0.05304485931992531,"0":0.03625566139817238,"1":0.04591182991862297,"2":0.056426506489515305,"3":0.042852871119976044,"4":0.06810418516397476,"5":0.07422996312379837,"6":0.03484737128019333,"7":0.039241041988134384,"8":0.05139022693037987,"9":0.043749529868364334,"10":0.030250554904341698},"Categ":5,"pred":"f","text":"awakenings due to itch over 16","id":34},{"score":0.08620596677064896,"probabs":{"11":0.08620596677064896,"12":0.04570329189300537,"13":0.0763409361243248,"14":0.03947865590453148,"15":0.05708197131752968,"16":0.048123329877853394,"17":0.06585317105054855,"18":0.04750034958124161,"19":0.05615234375,"0":0.03301052376627922,"1":0.05621261149644852,"2":0.058245766907930374,"3":0.02978755161166191,"4":0.05243918299674988,"5":0.054855793714523315,"6":0.03181491419672966,"7":0.03064623475074768,"8":0.04435751959681511,"9":0.050484549254179,"10":0.035705387592315674},"Categ":11,"pred":"l","text":"a Discontinued patients were considered as non-responders;","id":34},{"score":0.08329016715288162,"probabs":{"11":0.05852552875876427,"12":0.04928218200802803,"13":0.08329016715288162,"14":0.04763812571763992,"15":0.059170641005039215,"16":0.04568006843328476,"17":0.053653061389923096,"18":0.04153353348374367,"19":0.05460380017757416,"0":0.045776061713695526,"1":0.05843130871653557,"2":0.046965084969997406,"3":0.04383372887969017,"4":0.0456748865544796,"5":0.05467319115996361,"6":0.04252728819847107,"7":0.03679529204964638,"8":0.040985818952322006,"9":0.04432374984025955,"10":0.04663631692528725},"Categ":13,"pred":"n","text":"The proportion of patients achieving LDA with Olumiant + MTX was significantly higher than with adalimumab + MTX at week 12 and week 52","id":35},{"score":0.0786677822470665,"probabs":{"11":0.0734185054898262,"12":0.053138475865125656,"13":0.0645674467086792,"14":0.04427026957273483,"15":0.03951818495988846,"16":0.05541767179965973,"17":0.05795884504914284,"18":0.044276174157857895,"19":0.05284232273697853,"0":0.0347941555082798,"1":0.047643449157476425,"2":0.058559220284223557,"3":0.03778599575161934,"4":0.05978680029511452,"5":0.0786677822470665,"6":0.03158474713563919,"7":0.035702262073755264,"8":0.043942879885435104,"9":0.051764875650405884,"10":0.03435995429754257},"Categ":5,"pred":"f","text":" SEE ACCURACY IN ADULTS ?","id":31},{"score":0.07641265541315079,"probabs":{"11":0.06905900686979294,"12":0.04097370803356171,"13":0.07641265541315079,"14":0.04322277754545212,"15":0.057789236307144165,"16":0.04455627128481865,"17":0.06774531304836273,"18":0.04826144874095917,"19":0.0570475235581398,"0":0.04471467435359955,"1":0.06566658616065979,"2":0.0529228039085865,"3":0.04011021554470062,"4":0.047212518751621246,"5":0.055869825184345245,"6":0.03973971679806709,"7":0.028527729213237762,"8":0.03630654886364937,"9":0.04106844216585159,"10":0.04279303178191185},"Categ":13,"pred":"n","text":"50-56% of patients maintained remission (das28-hscrp 2.6)","id":32},{"score":0.07514258474111557,"probabs":{"11":0.05765023082494736,"12":0.05755647271871567,"13":0.07514258474111557,"14":0.04839545488357544,"15":0.046911340206861496,"16":0.04532558470964432,"17":0.05318426713347435,"18":0.041376255452632904,"19":0.06190388649702072,"0":0.042031336575746536,"1":0.05209396407008171,"2":0.049786075949668884,"3":0.04420416057109833,"4":0.06421200186014175,"5":0.0637398511171341,"6":0.04055355116724968,"7":0.03757360205054283,"8":0.036787837743759155,"9":0.04347537085413933,"10":0.03809618949890137},"Categ":13,"pred":"n","text":"Safety profile of baricitinib: All Bari dataset","id":34},{"score":0.07422996312379837,"probabs":{"11":0.06959327310323715,"12":0.06562454253435135,"13":0.07233278453350067,"14":0.04503995552659035,"15":0.040700286626815796,"16":0.04539260268211365,"17":0.04823519289493561,"18":0.03677678853273392,"19":0.05304485931992531,"0":0.03625566139817238,"1":0.04591182991862297,"2":0.056426506489515305,"3":0.042852871119976044,"4":0.06810418516397476,"5":0.07422996312379837,"6":0.03484737128019333,"7":0.039241041988134384,"8":0.05139022693037987,"9":0.043749529868364334,"10":0.030250554904341698},"Categ":5,"pred":"f","text":"awakenings due to itch over 16","id":34},{"score":0.08620596677064896,"probabs":{"11":0.08620596677064896,"12":0.04570329189300537,"13":0.0763409361243248,"14":0.03947865590453148,"15":0.05708197131752968,"16":0.048123329877853394,"17":0.06585317105054855,"18":0.04750034958124161,"19":0.05615234375,"0":0.03301052376627922,"1":0.05621261149644852,"2":0.058245766907930374,"3":0.02978755161166191,"4":0.05243918299674988,"5":0.054855793714523315,"6":0.03181491419672966,"7":0.03064623475074768,"8":0.04435751959681511,"9":0.050484549254179,"10":0.035705387592315674},"Categ":11,"pred":"l","text":"a Discontinued patients were considered as non-responders;","id":34},{"score":0.08329016715288162,"probabs":{"11":0.05852552875876427,"12":0.04928218200802803,"13":0.08329016715288162,"14":0.04763812571763992,"15":0.059170641005039215,"16":0.04568006843328476,"17":0.053653061389923096,"18":0.04153353348374367,"19":0.05460380017757416,"0":0.045776061713695526,"1":0.05843130871653557,"2":0.046965084969997406,"3":0.04383372887969017,"4":0.0456748865544796,"5":0.05467319115996361,"6":0.04252728819847107,"7":0.03679529204964638,"8":0.040985818952322006,"9":0.04432374984025955,"10":0.04663631692528725},"Categ":13,"pred":"n","text":"The proportion of patients achieving LDA with Olumiant + MTX was significantly higher than with adalimumab + MTX at week 12 and week 52","id":35},{"score":0.07161392271518707,"probabs":{"11":0.04035268723964691,"12":0.0406116247177124,"13":0.0643928274512291,"14":0.05198647826910019,"15":0.06164979189634323,"16":0.04718560725450516,"17":0.046289168298244476,"18":0.035153817385435104,"19":0.05515630543231964,"0":0.046835340559482574,"1":0.0294839758425951,"2":0.04554421827197075,"3":0.07161392271518707,"4":0.06186763942241669,"5":0.03344792127609253,"6":0.050818849354982376,"7":0.03936672955751419,"8":0.05561213567852974,"9":0.06607433408498764,"10":0.05655653029680252},"Categ":3,"pred":"d","text":" SEE ACCURACY IN ADULTS ?","id":31},{"score":0.16765771806240082,"probabs":{"11":0.03964937478303909,"12":0.0307786762714386,"13":0.16765771806240082,"14":0.0537228100001812,"15":0.027972500771284103,"16":0.03410036861896515,"17":0.0618680976331234,"18":0.05475699156522751,"19":0.046574413776397705,"0":0.03635594993829727,"1":0.07563778758049011,"2":0.0547444224357605,"3":0.04104313254356384,"4":0.02665884606540203,"5":0.04692531004548073,"6":0.028085438534617424,"7":0.062051597982645035,"8":0.03089076839387417,"9":0.041960708796978,"10":0.03856508806347847},"Categ":13,"pred":"n","text":"50-56% of patients maintained remission (das28-hscrp 2.6)","id":32},{"score":0.15550681948661804,"probabs":{"11":0.03936811536550522,"12":0.03349334001541138,"13":0.15550681948661804,"14":0.06039203330874443,"15":0.026066264137625694,"16":0.033811893314123154,"17":0.059158001095056534,"18":0.05527758225798607,"19":0.04501783475279808,"0":0.03612785413861275,"1":0.07790616899728775,"2":0.05150754004716873,"3":0.03515759855508804,"4":0.025282997637987137,"5":0.055473797023296356,"6":0.030911633744835854,"7":0.06860218942165375,"8":0.03120114468038082,"9":0.041929300874471664,"10":0.0378078855574131},"Categ":13,"pred":"n","text":"Safety profile of baricitinib: All Bari dataset","id":34},{"score":0.1593659222126007,"probabs":{"11":0.035464804619550705,"12":0.033989161252975464,"13":0.1593659222126007,"14":0.04602809250354767,"15":0.02928531914949417,"16":0.03307224065065384,"17":0.06733213365077972,"18":0.044933710247278214,"19":0.057174183428287506,"0":0.03866594657301903,"1":0.07209402322769165,"2":0.04763425514101982,"3":0.04568662494421005,"4":0.024899866431951523,"5":0.04650430381298065,"6":0.030026676133275032,"7":0.05320364236831665,"8":0.04166916757822037,"9":0.04342328757047653,"10":0.04954664781689644},"Categ":13,"pred":"n","text":"awakenings due to itch over 16","id":34},{"score":0.15153361856937408,"probabs":{"11":0.04391308128833771,"12":0.03673499822616577,"13":0.15153361856937408,"14":0.0446256659924984,"15":0.03344288095831871,"16":0.03624076023697853,"17":0.05632079020142555,"18":0.04608789458870888,"19":0.055711425840854645,"0":0.038944244384765625,"1":0.051296789199113846,"2":0.054851971566677094,"3":0.04556036368012428,"4":0.029197046533226967,"5":0.038453999906778336,"6":0.03314854949712753,"7":0.05106913298368454,"8":0.03985504060983658,"9":0.059651248157024384,"10":0.05336039885878563},"Categ":13,"pred":"n","text":"a Discontinued patients were considered as non-responders;","id":34},{"score":0.16927023231983185,"probabs":{"11":0.03820618987083435,"12":0.02950321137905121,"13":0.16927023231983185,"14":0.05062125623226166,"15":0.02767421491444111,"16":0.03556058183312416,"17":0.0639946237206459,"18":0.05535133555531502,"19":0.0547204352915287,"0":0.034825097769498825,"1":0.08999908715486526,"2":0.0443703792989254,"3":0.042134929448366165,"4":0.02220219373703003,"5":0.04599907621741295,"6":0.028274232521653175,"7":0.062090009450912476,"8":0.027946652844548225,"9":0.041481081396341324,"10":0.03577522933483124},"Categ":13,"pred":"n","text":"The proportion of patients achieving LDA with Olumiant + MTX was significantly higher than with adalimumab + MTX at week 12 and week 52","id":35},{"score":0.07161392271518707,"probabs":{"11":0.04035268723964691,"12":0.0406116247177124,"13":0.0643928274512291,"14":0.05198647826910019,"15":0.06164979189634323,"16":0.04718560725450516,"17":0.046289168298244476,"18":0.035153817385435104,"19":0.05515630543231964,"0":0.046835340559482574,"1":0.0294839758425951,"2":0.04554421827197075,"3":0.07161392271518707,"4":0.06186763942241669,"5":0.03344792127609253,"6":0.050818849354982376,"7":0.03936672955751419,"8":0.05561213567852974,"9":0.06607433408498764,"10":0.05655653029680252},"Categ":3,"pred":"d","text":" SEE ACCURACY IN ADULTS ?","id":31},{"score":0.16765771806240082,"probabs":{"11":0.03964937478303909,"12":0.0307786762714386,"13":0.16765771806240082,"14":0.0537228100001812,"15":0.027972500771284103,"16":0.03410036861896515,"17":0.0618680976331234,"18":0.05475699156522751,"19":0.046574413776397705,"0":0.03635594993829727,"1":0.07563778758049011,"2":0.0547444224357605,"3":0.04104313254356384,"4":0.02665884606540203,"5":0.04692531004548073,"6":0.028085438534617424,"7":0.062051597982645035,"8":0.03089076839387417,"9":0.041960708796978,"10":0.03856508806347847},"Categ":13,"pred":"n","text":"50-56% of patients maintained remission (das28-hscrp 2.6)","id":32},{"score":0.15550681948661804,"probabs":{"11":0.03936811536550522,"12":0.03349334001541138,"13":0.15550681948661804,"14":0.06039203330874443,"15":0.026066264137625694,"16":0.033811893314123154,"17":0.059158001095056534,"18":0.05527758225798607,"19":0.04501783475279808,"0":0.03612785413861275,"1":0.07790616899728775,"2":0.05150754004716873,"3":0.03515759855508804,"4":0.025282997637987137,"5":0.055473797023296356,"6":0.030911633744835854,"7":0.06860218942165375,"8":0.03120114468038082,"9":0.041929300874471664,"10":0.0378078855574131},"Categ":13,"pred":"n","text":"Safety profile of baricitinib: All Bari dataset","id":34},{"score":0.1593659222126007,"probabs":{"11":0.035464804619550705,"12":0.033989161252975464,"13":0.1593659222126007,"14":0.04602809250354767,"15":0.02928531914949417,"16":0.03307224065065384,"17":0.06733213365077972,"18":0.044933710247278214,"19":0.057174183428287506,"0":0.03866594657301903,"1":0.07209402322769165,"2":0.04763425514101982,"3":0.04568662494421005,"4":0.024899866431951523,"5":0.04650430381298065,"6":0.030026676133275032,"7":0.05320364236831665,"8":0.04166916757822037,"9":0.04342328757047653,"10":0.04954664781689644},"Categ":13,"pred":"n","text":"awakenings due to itch over 16","id":34},{"score":0.15153361856937408,"probabs":{"11":0.04391308128833771,"12":0.03673499822616577,"13":0.15153361856937408,"14":0.0446256659924984,"15":0.03344288095831871,"16":0.03624076023697853,"17":0.05632079020142555,"18":0.04608789458870888,"19":0.055711425840854645,"0":0.038944244384765625,"1":0.051296789199113846,"2":0.054851971566677094,"3":0.04556036368012428,"4":0.029197046533226967,"5":0.038453999906778336,"6":0.03314854949712753,"7":0.05106913298368454,"8":0.03985504060983658,"9":0.059651248157024384,"10":0.05336039885878563},"Categ":13,"pred":"n","text":"a Discontinued patients were considered as non-responders;","id":34},{"score":0.16927023231983185,"probabs":{"11":0.03820618987083435,"12":0.02950321137905121,"13":0.16927023231983185,"14":0.05062125623226166,"15":0.02767421491444111,"16":0.03556058183312416,"17":0.0639946237206459,"18":0.05535133555531502,"19":0.0547204352915287,"0":0.034825097769498825,"1":0.08999908715486526,"2":0.0443703792989254,"3":0.042134929448366165,"4":0.02220219373703003,"5":0.04599907621741295,"6":0.028274232521653175,"7":0.062090009450912476,"8":0.027946652844548225,"9":0.041481081396341324,"10":0.03577522933483124},"Categ":13,"pred":"n","text":"The proportion of patients achieving LDA with Olumiant + MTX was significantly higher than with adalimumab + MTX at week 12 and week 52","id":35},{"score":0.12634479999542236,"probabs":{"11":0.06247119605541229,"12":0.050037652254104614,"13":0.12634479999542236,"14":0.03795496001839638,"15":0.07269198447465897,"16":0.027252783998847008,"17":0.059340041130781174,"18":0.05814190208911896,"19":0.03066185675561428,"0":0.08061342686414719,"1":0.03354671597480774,"2":0.031937357038259506,"3":0.024694904685020447,"4":0.02910773642361164,"5":0.03633850812911987,"6":0.028910353779792786,"7":0.054589737206697464,"8":0.04773753881454468,"9":0.061636339873075485,"10":0.045990291982889175},"Categ":13,"pred":"n","text":" SEE ACCURACY IN ADULTS ?","id":31},{"score":0.1367417573928833,"probabs":{"11":0.050896987318992615,"12":0.03911470249295235,"13":0.1367417573928833,"14":0.044757746160030365,"15":0.07390004396438599,"16":0.027263911440968513,"17":0.078951396048069,"18":0.05710581690073013,"19":0.034363675862550735,"0":0.07538915425539017,"1":0.0314745232462883,"2":0.03379067778587341,"3":0.027952764183282852,"4":0.02624187432229519,"5":0.02886245958507061,"6":0.028559545055031776,"7":0.049197643995285034,"8":0.04167266935110092,"9":0.07045908272266388,"10":0.04330356791615486},"Categ":13,"pred":"n","text":"50-56% of patients maintained remission (das28-hscrp 2.6)","id":32},{"score":0.1276746690273285,"probabs":{"11":0.0699823871254921,"12":0.06492919474840164,"13":0.1276746690273285,"14":0.06334757804870605,"15":0.06472764164209366,"16":0.023341771215200424,"17":0.050270646810531616,"18":0.04907143861055374,"19":0.040460627526044846,"0":0.05867870897054672,"1":0.039756033569574356,"2":0.03360401839017868,"3":0.032999467104673386,"4":0.03317135199904442,"5":0.039395805448293686,"6":0.03935551643371582,"7":0.053492527455091476,"8":0.04676760360598564,"9":0.039962515234947205,"10":0.02901049703359604},"Categ":13,"pred":"n","text":"Safety profile of baricitinib: All Bari dataset","id":34},{"score":0.13062074780464172,"probabs":{"11":0.050332002341747284,"12":0.03898337110877037,"13":0.13062074780464172,"14":0.04934326186776161,"15":0.05966152250766754,"16":0.02854563295841217,"17":0.06559865921735764,"18":0.05329029634594917,"19":0.0334944985806942,"0":0.07434457540512085,"1":0.03296084329485893,"2":0.03008386306464672,"3":0.029992472380399704,"4":0.022911272943019867,"5":0.030816378071904182,"6":0.03294844180345535,"7":0.05940934270620346,"8":0.04975578561425209,"9":0.07614672183990479,"10":0.0507601723074913},"Categ":13,"pred":"n","text":"awakenings due to itch over 16","id":34},{"score":0.1272536665201187,"probabs":{"11":0.06301098316907883,"12":0.037651415914297104,"13":0.1272536665201187,"14":0.04191889613866806,"15":0.06258834898471832,"16":0.025729723274707794,"17":0.06925690174102783,"18":0.06509364396333694,"19":0.0289076566696167,"0":0.08100893348455429,"1":0.03217019513249397,"2":0.03152613714337349,"3":0.025309674441814423,"4":0.02690909057855606,"5":0.03174332529306412,"6":0.027309702709317207,"7":0.05365825444459915,"8":0.04302874952554703,"9":0.0740022137761116,"10":0.051922425627708435},"Categ":13,"pred":"n","text":"a Discontinued patients were considered as non-responders;","id":34},{"score":0.13336044549942017,"probabs":{"11":0.05540933832526207,"12":0.04142050817608833,"13":0.13336044549942017,"14":0.05768578499555588,"15":0.07768985629081726,"16":0.026753198355436325,"17":0.06462326645851135,"18":0.05655119940638542,"19":0.04110392555594444,"0":0.06926074624061584,"1":0.03136739879846573,"2":0.033163025975227356,"3":0.028852013871073723,"4":0.030042679980397224,"5":0.03045685403048992,"6":0.038417547941207886,"7":0.04936453327536583,"8":0.046603377908468246,"9":0.05401873216032982,"10":0.03385550528764725},"Categ":13,"pred":"n","text":"The proportion of patients achieving LDA with Olumiant + MTX was significantly higher than with adalimumab + MTX at week 12 and week 52","id":35},{"score":0.12634479999542236,"probabs":{"11":0.06247119605541229,"12":0.050037652254104614,"13":0.12634479999542236,"14":0.03795496001839638,"15":0.07269198447465897,"16":0.027252783998847008,"17":0.059340041130781174,"18":0.05814190208911896,"19":0.03066185675561428,"0":0.08061342686414719,"1":0.03354671597480774,"2":0.031937357038259506,"3":0.024694904685020447,"4":0.02910773642361164,"5":0.03633850812911987,"6":0.028910353779792786,"7":0.054589737206697464,"8":0.04773753881454468,"9":0.061636339873075485,"10":0.045990291982889175},"Categ":13,"pred":"n","text":" SEE ACCURACY IN ADULTS ?","id":31},{"score":0.1367417573928833,"probabs":{"11":0.050896987318992615,"12":0.03911470249295235,"13":0.1367417573928833,"14":0.044757746160030365,"15":0.07390004396438599,"16":0.027263911440968513,"17":0.078951396048069,"18":0.05710581690073013,"19":0.034363675862550735,"0":0.07538915425539017,"1":0.0314745232462883,"2":0.03379067778587341,"3":0.027952764183282852,"4":0.02624187432229519,"5":0.02886245958507061,"6":0.028559545055031776,"7":0.049197643995285034,"8":0.04167266935110092,"9":0.07045908272266388,"10":0.04330356791615486},"Categ":13,"pred":"n","text":"50-56% of patients maintained remission (das28-hscrp 2.6)","id":32},{"score":0.1276746690273285,"probabs":{"11":0.0699823871254921,"12":0.06492919474840164,"13":0.1276746690273285,"14":0.06334757804870605,"15":0.06472764164209366,"16":0.023341771215200424,"17":0.050270646810531616,"18":0.04907143861055374,"19":0.040460627526044846,"0":0.05867870897054672,"1":0.039756033569574356,"2":0.03360401839017868,"3":0.032999467104673386,"4":0.03317135199904442,"5":0.039395805448293686,"6":0.03935551643371582,"7":0.053492527455091476,"8":0.04676760360598564,"9":0.039962515234947205,"10":0.02901049703359604},"Categ":13,"pred":"n","text":"Safety profile of baricitinib: All Bari dataset","id":34},{"score":0.13062074780464172,"probabs":{"11":0.050332002341747284,"12":0.03898337110877037,"13":0.13062074780464172,"14":0.04934326186776161,"15":0.05966152250766754,"16":0.02854563295841217,"17":0.06559865921735764,"18":0.05329029634594917,"19":0.0334944985806942,"0":0.07434457540512085,"1":0.03296084329485893,"2":0.03008386306464672,"3":0.029992472380399704,"4":0.022911272943019867,"5":0.030816378071904182,"6":0.03294844180345535,"7":0.05940934270620346,"8":0.04975578561425209,"9":0.07614672183990479,"10":0.0507601723074913},"Categ":13,"pred":"n","text":"awakenings due to itch over 16","id":34},{"score":0.1272536665201187,"probabs":{"11":0.06301098316907883,"12":0.037651415914297104,"13":0.1272536665201187,"14":0.04191889613866806,"15":0.06258834898471832,"16":0.025729723274707794,"17":0.06925690174102783,"18":0.06509364396333694,"19":0.0289076566696167,"0":0.08100893348455429,"1":0.03217019513249397,"2":0.03152613714337349,"3":0.025309674441814423,"4":0.02690909057855606,"5":0.03174332529306412,"6":0.027309702709317207,"7":0.05365825444459915,"8":0.04302874952554703,"9":0.0740022137761116,"10":0.051922425627708435},"Categ":13,"pred":"n","text":"a Discontinued patients were considered as non-responders;","id":34},{"score":0.13336044549942017,"probabs":{"11":0.05540933832526207,"12":0.04142050817608833,"13":0.13336044549942017,"14":0.05768578499555588,"15":0.07768985629081726,"16":0.026753198355436325,"17":0.06462326645851135,"18":0.05655119940638542,"19":0.04110392555594444,"0":0.06926074624061584,"1":0.03136739879846573,"2":0.033163025975227356,"3":0.028852013871073723,"4":0.030042679980397224,"5":0.03045685403048992,"6":0.038417547941207886,"7":0.04936453327536583,"8":0.046603377908468246,"9":0.05401873216032982,"10":0.03385550528764725},"Categ":13,"pred":"n","text":"The proportion of patients achieving LDA with Olumiant + MTX was significantly higher than with adalimumab + MTX at week 12 and week 52","id":35}]

In [59]:
len(v)

36

In [60]:
v[1]

{'score': 0.07641265541315079,
 'probabs': {'11': 0.06905900686979294,
  '12': 0.04097370803356171,
  '13': 0.07641265541315079,
  '14': 0.04322277754545212,
  '15': 0.057789236307144165,
  '16': 0.04455627128481865,
  '17': 0.06774531304836273,
  '18': 0.04826144874095917,
  '19': 0.0570475235581398,
  '0': 0.04471467435359955,
  '1': 0.06566658616065979,
  '2': 0.0529228039085865,
  '3': 0.04011021554470062,
  '4': 0.047212518751621246,
  '5': 0.055869825184345245,
  '6': 0.03973971679806709,
  '7': 0.028527729213237762,
  '8': 0.03630654886364937,
  '9': 0.04106844216585159,
  '10': 0.04279303178191185},
 'Categ': 13,
 'pred': 'n',
 'text': '50-56% of patients maintained remission (das28-hscrp 2.6)',
 'id': 32}

In [61]:
len([
    {"id": 31, "text": " SEE ACCURACY IN ADULTS →"
    },
    {"id": 32, "text": "50-56% of patients maintained remission (das28-hscrp 2.6)"
    },
    {"id": 34, "text": "Safety profile of baricitinib: All Bari dataset"
    },
    {"id":34,"text": "awakenings due to itch over 16"
    },
    {"id":34,"text": "a Discontinued patients were considered as non-responders;"
    },
    {"id":35,"text": "The proportion of patients achieving LDA with Olumiant + MTX was significantly higher than with adalimumab + MTX at week 12 and week 52"
    },
    {"id": 31, "text": " SEE ACCURACY IN ADULTS →"
    },
    {"id": 32, "text": "50-56% of patients maintained remission (das28-hscrp 2.6)"
    },
    {"id": 34, "text": "Safety profile of baricitinib: All Bari dataset"
    },
    {"id":34,"text": "awakenings due to itch over 16"
    },
    {"id":34,"text": "a Discontinued patients were considered as non-responders;"
    },
    {"id":35,"text": "The proportion of patients achieving LDA with Olumiant + MTX was significantly higher than with adalimumab + MTX at week 12 and week 52"
    }
])

12

In [62]:
len([{"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"},
   {"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"},
  {"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"},
   {"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"},
  {"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"},
   {"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"},
  {"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"},
   {"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"},
  {"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"},
   {"text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
  "id": "1"}
 ]
)

10

In [91]:
import requests
import json

url = "https://hxr7tn32b0.execute-api.us-east-2.amazonaws.com/demo1"

payload = json.dumps([
  {
    "text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better than his regular season stats he is also a lotfo fun to watch in the playoffs bowman should let jagr \nhave a lot offun in the next couple of games since the pens are going to beat the pulp out of jerseyanyway i was very \ndisappointed not to see the islanders lose the finalregular season game pens rule",
    "id": "1"
  }
])
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


[{"score": 0.16138790547847748, "Categ": 10, "pred": "k", "probabs": {"0": 0.045817021280527115, "1": 0.024176303297281265, "2": 0.03678528219461441, "3": 0.07134871929883957, "4": 0.048216186463832855, "5": 0.03441502898931503, "6": 0.05540027469396591, "7": 0.05339713767170906, "8": 0.04719889909029007, "9": 0.07552622258663177, "10": 0.16138790547847748, "11": 0.026280518621206284, "12": 0.059216320514678955, "13": 0.05626598373055458, "14": 0.054571881890296936, "15": 0.03751858323812485, "16": 0.03490070998668671, "17": 0.02578582800924778, "18": 0.030874980613589287, "19": 0.020916156470775604}, "text": "i am sure some bashers of pens fans are pretty confused about the lackof any kind of posts about the recent pens \nmassacre of the devils actuallyi am  bit puzzled too and a bit relieved however i am going to put an endto nonpittsburghers \nrelief with a bit of praise for the pens man theyare killing those devils worse than i thought jagr just showed you why \nhe is much better t